In [1]:
import pandas as pd
import sqlite3
import numpy as np

In [4]:
def clean_and_store_csv(csv_file, db_file, table_name, primary_key=None, foreign_keys=None, dropna=True, fillna_value=None):
    """
    Reads a CSV file, performs basic data cleaning, and stores the cleaned data into an SQLite3 database.

    Args:
    csv_file (str): The path to the CSV file.
    db_file (str): The path to the SQLite3 database file.
    table_name (str): The name of the table where the data will be stored.

    Returns:
    None
    """
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file,na_values='\\N')

    # Replace '\n' with NaN
    df.replace(to_replace=r'\\n', value=np.nan, regex=True, inplace=True)

    # Basic data cleaning operations
    
    if dropna:
        df.dropna(inplace=True)  # Drop rows with any NaN values
    elif fillna_value is not None:
        df.fillna(fillna_value, inplace=True)  # Fill NaN values with the specified value



    # Connect to the SQLite3 database 
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    # Drop the table if it already exists
    cursor.execute(f"DROP TABLE IF EXISTS {table_name}")

    # Map Datatype in SQLite database from dataframe dtype
    dtype_mapping = {
        'object': 'TEXT',
        'int64': 'INTEGER',
        'float64': 'REAL',
        'datetime64[ns]': 'TEXT'  
    }

    columns = df.columns
    col_defs = []
    
    for col in columns:
        col_type = dtype_mapping[str(df[col].dtype)]
        col_defs.append(f"{col} {col_type}")   

    if primary_key:
        col_defs += f", PRIMARY KEY ({primary_key})"

    if foreign_keys:
        for col, ref in foreign_keys.items():
            col_defs += f", FOREIGN KEY ({col}) REFERENCES {ref}"

    create_table_sql = f"CREATE TABLE {table_name} ({col_defs})"
    cursor.execute(create_table_sql)
    
    # Write the cleaned data to the specified table
    
    df.to_sql(table_name, conn, if_exists='replace', index=False)

    # Close the database connection
    conn.close()

    print(f"Cleaned data from {csv_file} has been successfully stored in {db_file} in the {table_name} table.")



In [6]:
def query_database(db_file, table_name, primary_key, key_value):
    """
    Reads data from the SQLite3 database and performs a simple SQL query using the primary key.

    Args:
    db_file (str): The path to the SQLite3 database file.
    table_name (str): The name of the table to query.
    primary_key (str): The column to use as the primary key for the query.
    key_value (any): The value of the primary key to search for.

    Returns:
    pd.DataFrame: A DataFrame containing the query result.
    """
    # Connect to the SQLite3 database
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Perform the query
    query = f"SELECT * FROM {table_name} WHERE {primary_key} = ?"
    cursor.execute(query, (key_value,))
    rows = cursor.fetchall()

    # Fetch the column names
    column_names = [description[0] for description in cursor.description]

    # Close the database connection
    conn.close()

    # Convert the result to a pandas DataFrame
    df = pd.DataFrame(rows, columns=column_names)

    return df


In [15]:
def add_index_column_to_csv(csv_file, index_column_name, output_csv_file):
    """
    Reads a CSV file, adds an index column to the leftmost column with the supplied column name,
    and writes the updated data back to a new CSV file.

    Args:
    csv_file (str): The path to the input CSV file.
    index_column_name (str): The name for the new index column.
    output_csv_file (str): The path to the output CSV file.

    Returns:
    None
    """
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)
    
    # Create the index column
    df.insert(0, index_column_name, range(1, len(df) + 1))

    # Write the updated DataFrame to a new CSV file
    df.to_csv(output_csv_file, index=False)

    print(f"Index column '{index_column_name}' has been added and written to {output_csv_file}")




In [7]:
csv_file = './dataset/circuits.csv'  
db_file = 'f1_database.db'  
table_name = 'circuits_table' 
primary_key = 'circuitId' 
clean_and_store_csv(csv_file, db_file, table_name, primary_key,dropna=False,fillna_value='N/A' )

Cleaned data from ./dataset/circuits.csv has been successfully stored in williams_f1_database.db in the circuits_table table.


In [9]:
csv_file = './dataset/races.csv' 
db_file = 'f1_database.db' 
table_name = 'races_table'
primary_key = 'raceId' 
foreign_keys = {'circuitId': 'circuits_table(circuitId)'}
clean_and_store_csv(csv_file, db_file, table_name, primary_key,dropna=False,fillna_value='N/A')

Cleaned data from ./dataset/races.csv has been successfully stored in williams_f1_database.db in the races_table table.


In [10]:
db_file = 'f1_database.db' 
table_name = 'races_table'  
primary_key = 'raceId'   
key_value = 1062        
result_df = query_database(db_file, table_name, primary_key, key_value)

In [12]:
csv_file = './dataset/drivers.csv'  
db_file = 'f1_database.db'    
table_name = 'drivers_table' 
primary_key = 'driverId'
clean_and_store_csv(csv_file, db_file, table_name, primary_key,dropna=False,fillna_value='NaN')

Cleaned data from ./dataset/drivers.csv has been successfully stored in williams_f1_database.db in the drivers_table table.


C:\Users\amitv\AppData\Local\Temp\ipykernel_18100\825136183.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NaN' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna(fillna_value, inplace=True)  # Fill NaN values with the specified value


In [13]:
csv_file = './dataset/driver_standings.csv'  
db_file = 'f1_database.db'    
table_name = 'driver_standing_table' 
primary_key = 'driverStandingsId' 
foreign_keys = {'raceId': 'races_table(raceId)', 'driverId': 'drivers_table(driverId)'}  # Example foreign key definition #Name of column for foreign key
clean_and_store_csv(csv_file, db_file, table_name, primary_key,foreign_keys,dropna=False,fillna_value='N/A')

Cleaned data from ./dataset/driver_standings.csv has been successfully stored in williams_f1_database.db in the driver_standing_table table.


In [17]:

csv_file = './dataset/lap_times.csv'  
index_column_name = 'laptimesId'  
output_csv_file = './dataset/lap_times.csv' 

add_index_column_to_csv(csv_file, index_column_name, output_csv_file)

Index column 'laptimesId' has been added and written to ./dataset/lap_times.csv


In [18]:
csv_file = './dataset/lap_times.csv' 
db_file = 'f1_database.db'    
table_name = 'lap_times' 
primary_key = 'laptimesId'
foreign_keys = {'raceId': 'races_table(raceId)', 'driverId': 'drivers_table(driverId)'}  
clean_and_store_csv(csv_file, db_file, table_name, primary_key,dropna=False,fillna_value='N/A')

Cleaned data from ./dataset/lap_times.csv has been successfully stored in williams_f1_database.db in the lap_times table.
